# DACS Elevator Radio Producer

In [2]:
# Install all relied libraries
#!pip install -r requirements.txt

# Initialize
from data.base import store_lyrics

## Get data from billboard top 100
If this year's data is stored then it won't grab again. You can directly use them.

In [3]:
store_lyrics(year = None)

Illegal input year. Setting year to be the current.
Searching for "Heat Waves" by Glass Animals...
Done.
Searching for "As It Was" by Harry Styles...
Done.
Searching for "Stay" by The Kid LAROI & Justin Bieber...
Done.
Searching for "Easy On Me" by Adele...
Done.
Searching for "Shivers" by Ed Sheeran...
Done.
Searching for "First Class" by Jack Harlow...
Done.
Searching for "Big Energy" by Latto...
Done.
Searching for "Ghost" by Justin Bieber...
Done.
Searching for "Super Gremlin" by Kodak Black...
Done.
Searching for "Cold Heart (PNAU Remix)" by Elton John & Dua Lipa...
Done.
Searching for "Wait For U" by Future Featuring Drake & Tems...
Done.
Searching for "About Damn Time" by Lizzo...
Done.
Searching for "Bad Habits" by Ed Sheeran...
Done.
Searching for "Thats What I Want" by Lil Nas X...
Done.
Searching for "Enemy" by Imagine Dragons X JID...
Done.
Searching for "Industry Baby" by Lil Nas X & Jack Harlow...
Done.
Searching for "abcdefu" by GAYLE...
Done.
Searching for "Need To Know

## Get data from a playlist from spotify
### How to find playlist ID:
![playlist id](imgs/playlist_id.png)

Or if you share the playlist with link, find the string before `?si=`:  
https://open.spotify.com/playlist/**37i9dQZF1DX5Ejj0EkURtP**?si=a1e0243dd67c4cc3


In [4]:
store_lyrics(playlist_id="1q2ztYCQCBrq14wFEJKOB8")

Searching for "Apocalypse" by Cigarettes After Sex...
Done.
Searching for "Sunsetz" by Cigarettes After Sex...
Done.
Searching for "Cry" by Cigarettes After Sex...
Done.
Searching for "K." by Cigarettes After Sex...
Done.
Searching for "My Jinji" by Sunset Rollercoaster...
Done.
Searching for "I Know You Know I Love You" by Sunset Rollercoaster...
Done.
Searching for "Jellyfish" by Sunset Rollercoaster...
Done.
Searching for "我是一隻魚 I'm a Fish" by Sunset Rollercoaster...
Done.
Searching for "Blues" by Sunset Rollercoaster...
Done.
Searching for "Welcome To" by Sunset Rollercoaster...
Specified song does not contain lyrics. Rejecting.
There's error when getting lyrcis of 'Welcome To' by Sunset Rollercoaster
'NoneType' object has no attribute 'lyrics'
Searching for "Vanilla" by Sunset Rollercoaster...
Done.
Searching for "Villa" by Sunset Rollercoaster...
Done.
Searching for "I Wanna Be Yours" by Arctic Monkeys...
Done.
Searching for "I Wanna Be Adored - Remastered" by The Stone Roses...
